In [6]:
using CSV
using DataFrames

#Define function create symmetric matrix costs
function create_symmetric_matrix(x::Vector{T}) where T<:Number
    n = length(x)
    A = zeros(Float64, n, n)
    A[1, 2:end] .= x[2:end]
    A[2:end, 1] .= x[2:end]
    A[1:n+1:end] .= 0

    for i in 2:n
        for j in 2:n
            A[i, j] = (x[i-1] + x[j-1]) / 2
        end
    end
    
    A[1:n+1:end] .= 0
    
    return A
end

data = DataFrame(CSV.File("sorted_rec.csv"))
scores = data[!, "score"]

costs = create_symmetric_matrix(scores)
objects = data[!, "ID"]

times = DataFrame(CSV.File("times.csv", header = false))
waits = DataFrame(CSV.File("waits.csv", header = false))
distances =  DataFrame(CSV.File("distances.csv", header = false))


Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21,Column22,Column23,Column24,Column25,Column26,Column27,Column28,Column29,Column30,Column31,Column32,Column33,Column34,Column35,Column36,Column37,Column38,Column39,Column40,Column41,Column42,Column43,Column44,Column45,Column46,Column47,Column48,Column49,Column50,Column51,Column52,Column53,Column54,Column55,Column56,Column57,Column58,Column59,Column60,Column61,Column62,Column63,Column64,Column65,Column66,Column67,Column68,Column69,Column70,Column71,Column72,Column73,Column74,Column75,Column76,Column77,Column78,Column79,Column80,Column81,Column82,Column83,Column84,Column85,Column86,Column87,Column88,Column89,Column90,Column91,Column92,Column93,Column94,Column95,Column96,Column97,Column98,Column99,Column100,⋯
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,⋯
2,0.0,36.5926,20.1581,38.7861,21.123,21.1152,15.4298,34.8286,23.8918,9.88304,30.8505,32.2143,31.9324,13.141,20.1379,24.8829,21.7779,33.5034,35.7179,23.2397,27.259,37.4495,6.236,25.0034,26.1725,35.9803,27.2681,27.0585,19.4886,39.2585,20.7245,18.6148,28.9819,16.823,16.9269,7.78829,39.4129,31.8913,35.5707,23.457,20.6197,26.6475,20.8375,21.868,39.699,21.5307,25.6285,14.4565,19.8836,28.1204,26.8432,4.71669,38.8432,35.6816,16.7215,36.3422,5.03367,10.22,27.4699,9.74357,36.8012,29.1305,21.4666,24.1861,23.5693,33.0643,27.6193,26.9081,27.465,29.9384,12.8221,6.40355,39.5125,25.4553,26.2654,34.7301,23.8296,31.8377,41.2552,21.5707,22.3143,31.4893,29.6824,30.6001,36.9436,32.8111,33.9417,40.5872,4.54855,20.9494,35.9233,40.721,37.6199,28.7549,35.6579,36.17,21.3355,11.9059,35.7814,35.3941,⋯
3,36.5926,0.0,33.5124,48.1405,30.4773,19.4586,51.685,44.1829,33.2461,34.4121,40.2048,41.5686,41.2867,26.4953,33.4922,12.2872,35.1322,42.8578,45.0722,32.594,36.6133,46.8038,42.5633,34.3577,37.5268,45.3346,38.6224,36.4128,30.8429,48.6128,34.0789,20.6196,38.3362,26.1774,30.2812,36.7513,48.7672,41.2456,5.34597,32.8113,29.974,36.0019,30.1918,31.2223,49.0533,17.1756,38.9828,25.8108,29.2379,39.4747,36.1975,41.0525,48.1975,49.0359,30.0758,3.17311,38.3902,46.7581,36.8243,46.0147,46.1555,38.4848,32.8209,33.5404,13.586,44.4186,9.28336,36.2624,38.8193,39.2927,49.3897,40.1045,48.8668,34.8096,37.6197,44.0844,33.1839,41.192,50.6095,34.925,14.3271,42.8436,39.0367,7.09055,50.2979,42.1654,5.54383,49.9415,37.1981,34.3037,45.2776,50.0753,46.9743,42.1092,45.0122,49.5243,32.6898,25.2603,47.1358,44.7484,⋯
4,20.1581,33.5124,0.0,35.7059,18.0427,18.035,35.2505,31.7483,20.8115,17.9775,27.7702,29.1341,28.8522,7.21335,6.4292,21.8026,9.4291,30.4232,32.6376,20.1594,24.1788,34.3692,26.1287,21.9231,23.0923,32.9001,24.1878,23.9783,16.4084

In [36]:
# Define the generate start route function
function generateRoute(exhibits, max_exhibits)
    n = length(exhibits)
    route = zeros(Int, min(n, max_exhibits))
    visited = falses(n)
  
    # Ensure the first exhibit is included in the route
    route[1] = 1
    visited[1] = true

    for i in 2:min(n, max_exhibits)
        unvisited = filter(x -> !visited[x], 1:n)
        index = rand(1:length(unvisited))
        route[i] = unvisited[index]
        visited[route[i]] = true
    end

    # Ensure the last exhibit is included in the route
    if n > max_exhibits
        route[end] = n
        visited[n] = true
    end

    return route
end

max_time = 7200
max_iterations = 150

# Call the generate start route function
start_solution = generateRoute(objects, 100)
node_count = length(start_solution)
start_solution

100-element Vector{Int64}:
    1
 1720
 1509
  142
  654
 1110
  424
 1444
   88
  973
 1041
  779
   42
    ⋮
 1219
 1546
 2057
 2001
  459
 1512
 2142
 2000
 1629
 1389
  136
 2200

In [37]:
using Random

function ILS(distances::DataFrame, times::DataFrame, costs::Matrix{Float64}, waits::DataFrame, node_count::Int64, max_time::Int64, max_iterations::Int64, start_solution::Vector{Int64})
    # Define a local search function
    function local_search(current_solution, current_time, current_cost)
        # Generate a random neighboring solution by swapping two randomly selected nodes
        next_solution = copy(current_solution)
        idx1 = rand(2:node_count)
        idx2 = rand(2:node_count)
        while idx2 == idx1 
            idx2 = rand(2:node_count)
        end
        next_solution[idx1], next_solution[idx2] = next_solution[idx2], next_solution[idx1]

        # Compute the time and cost of the new solution
        next_time = current_time
        next_cost = current_cost
        for i in 1:(node_count-1)
            next_time += times[next_solution[i], next_solution[i+1]]
            next_time += waits[next_solution[i], next_solution[i+1]]
            next_cost += costs[next_solution[i], next_solution[i+1]]
        end

        # Keep the new solution if it is better than the current solution
        if next_time <= max_time && next_cost > current_cost
            return next_solution, next_time, next_cost
        else
            return current_solution, current_time, current_cost
        end
    end
    
    

    # Initialize the best solution with the first node visited first
    best_solution = copy(start_solution)
    best_time = 0
    best_cost = 0

    # Initialize the current solution
    current_solution = best_solution
    current_time = best_time
    current_cost = best_cost
    
    
    # Create an array to store the total cost of each iteration
    total_costs = []

    # Start the iteration
    for i in 1:max_iterations
        # Use the local search function to get the next solution
        current_solution, current_time, current_cost = local_search(current_solution, current_time, current_cost)
        # Keep the best solution
        if current_cost > best_cost
            best_solution = copy(current_solution)
            best_time = current_time
            best_cost = current_cost
        end
        # Store the best cost of each iteration
        push!(total_costs, best_cost)
    end
    

    # Return the best solution
    return best_solution, best_time, best_cost
end

ILS (generic function with 1 method)

In [38]:
# Call the solve function
@time begin
  #code
solution, time, cost = ILS(distances, times, costs, waits, node_count, max_time, max_iterations, start_solution)
solution, cost
end

  0.542955 seconds (142.07 k allocations: 4.466 MiB, 54.95% compilation time)


([1, 1720, 1509, 301, 654, 1110, 424, 1444, 88, 973  …  2057, 2001, 459, 1512, 2142, 2000, 1629, 1389, 136, 2200], 10244.248716648455)